# Lab 3: Voxelwise encoding models
In this notebook, we'll develop voxelwise encoding models (VEMs) for mapping embeddings from a language model onto human brain activity during natural language comprehension. We'll first construct VEMs for one set of language features, then use banded ridge regression to compare to different sets of langauge features.  Encoding models are estimated using banded ridge regression with cross-validation—this allows us to predict brain activity from word embeddings for left-out segments of data.

*Acknowledgments:* This tutorial draws heavily on work by co-author Zaid Zada (e.g. code from [Zada et al., 2023](https://doi.org/10.1016/j.neuron.2024.06.025), [2025](https://doi.org/10.1016/j.neuron.2025.11.004)) as well as Gallant Lab's [voxelwise modeling tutorials](https://gallantlab.org/voxelwise_tutorials/index.html) ([Dupré La Tour et al., 2022](https://doi.org/10.1016/j.neuroimage.2022.119728), [2025](https://doi.org/10.1162/imag_a_00575)).

---

First, you'll need to install a few more Python packages for new software used in this tutorial. Run the following lines in the command line. You may want to shut down your Jupyter Lab entirely and restart to make sure the progress bar widget works properly.

`conda install -c conda-forge gensim transformers accelerate ipywidgets`<br>
`pip install voxelwise_tutorials`

In [ ]:
from os.path import exists
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm.notebook import tqdm

# Explicitly set Mac locale character type to English
!export LC_CTYPE="en_US.UTF-8"

## Natural language comprehension fMRI dataset
In our first example, we'll use fMRI data collected for a single subject listening to a spoken story called "[I Knew You Were Black](https://themoth.org/stories/i-knew-you-were-black)" by Carol Daniel. This dataset should be quick (less than a minute) to download! These data are part of the publicly available [Narratives](https://github.com/snastase/narratives) collection ([Nastase et al., 2021](https://doi.org/10.1038/s41597-021-01033-3)). This dataset has been preprocessed using [fMRIPrep](https://fmriprep.org/en/stable/) with confound regression in [AFNI](https://afni.nimh.nih.gov/). The functional data have been spatially normalized to a template in MNI space.

In [ ]:
# Download data from Zenodo if we don't already have it
if not exists('encling-data.tgz'):
    # Use wget instead on a Linux machine
    # or download by hand: https://zenodo.org/records/8216229
    !curl -O https://zenodo.org/records/8216229/files/encling-data.tgz
    !tar -xvzf encling-data.tgz

To reduce computational demands, we compute parcel-wise VEMs using a cortical parcellation containing 400 parcels ([Schaefer et al., 2018](https://doi.org/10.1093/cercor/bhx179)). `NiftiLabelsMasker` provides an easy way to extract the mean BOLD time series from each parcel in the atlas.

In [ ]:
from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.maskers import NiftiLabelsMasker

# Preprocessed fMRI data from Narratives
func_fn = ('encling-data/sub-284_task-black_space-MNI152NLin2009cAsym_res-native_desc-clean_bold.nii.gz')

# Fetch Schaefer atlas with 400 parcels and 17 Yeo networks
n_parcels = 400
atlas = fetch_atlas_schaefer_2018(n_rois=n_parcels, yeo_networks=17, resolution_mm=2)

# Initialize labels masker with atlas parcels
masker = NiftiLabelsMasker(atlas.maps, labels=atlas.labels)

Use the masker's `.fit_transform()` method to extract parcel-level time series from `func_fn`. In my code, I called the output object `func_parcels`. Inspect the shape of your `func_parcels` and make sure you know what each dimension corresponds to.

In [ ]:
# Fit masker to extract mean time series for parcels:


To orient ourselves, let's plot the time series from an example parcel in anterior superior temporal cortex. Use `example_parcel = 195` to index the time series from `func_parcels` and plot using matplotlib's `plot()` function.

In [ ]:
# Plot the time series for an example parcel 195:


To transform parcel-level data back into a brain image, we'll use the `masker`'s `.inverse_transform()` method; this will allow us to visualize the location of the example parcel on the brain. Create an empty array of zeros the same shape as `func_parcels` and change the value at the `example_parcel` index to `1`. Then, use `.inverse_transform()` and plot the resulting brain image using `plot_stat_map` from `nilearn.plotting`.

In [ ]:
# Construct simple atlas map with 1 at parcel 195:

# Invert masker transform to project onto brain:

# Plot parcel on MNI atlas:


## Extracting word embeddings
In the following sections, we extract two types of vectors—called "word embeddings"—capturing the meaning of the words in our transcript. In both cases, words are encoded as vectors of continuous numeric values in a high-dimensional embedding space where each dimension corresponds to an internal feature of the model. Words that are similar to each other are located nearing to each other in this embedding space.

### Static word embeddings
In the first case, we'll retrieve static, noncontextual (lexical) word embeddings from a pre-trained model called word2vec ([Mikolov et al., 2013](https://doi.org/10.48550/arXiv.1301.3781)). This word2vec model was (pre)trained the Google News corpus containing approximately 100 billion words. We use a 300-dimensional word2vec model; that is, the hidden layer contains 300 units, resulting in 300-long word vectors. The word2vec embeddings are considered static embeddings because they capture the global meaning of a given word (based on its co-occurrence with other words), and each occurrence of a given word receives the same embedding regardless of the surrounding context. The model is about 2 GB in size, so it may take some time to download.

In [ ]:
import gensim.downloader

# Download 300-dimensional word2vec embeddings
model_name = 'word2vec-google-news-300'
n_features = 300

model = gensim.downloader.load(model_name)

This model is structured like a dictionary with strings (words) as keys and vectors (embeddings) as values. Try looking up an example word and inspect the shape of the corresponding embedding.

In [ ]:
# Check shape of embedding for example word:


Next, for every word in our transcript, we'll retrieve the corresponding embedding. We'll ignore any words that are not found in the word2vec vocabulary. We'll add these embeddings into our transcript and resave the transcript for easier loading later on. Make sure you understand what each line of code is doing in the cell below.

In [ ]:
# Load in transcript CSV file
transcript_f = 'encling-data/black_transcript.csv'
transcript_w2v = pd.read_csv(transcript_f)

# Convert words to lowercase
transcript_w2v['word'] = transcript_w2v.word.str.lower()

# Function to extract embeddings if available
def get_vector(word):
    if word in model.key_to_index:
        return model.get_vector(word, norm=True).astype(np.float32)
    return np.nan

# Extract embedding for each word
transcript_w2v['embedding'] = transcript_w2v.word.apply(get_vector)  
transcript_w2v = transcript_w2v.astype({'onset': 'float32', 'offset': 'float32'}, copy=False)

# Print out words not found in vocabulary
print(f'{(transcript_w2v.embedding.isna()).sum()} words not found:')
print(transcript_w2v.word[transcript_w2v.embedding.isna()].value_counts())

# Save transcript with embeddings using pickle
with open('black_w2v.pkl', 'wb') as f:
    pickle.dump(transcript_w2v, f)

In [ ]:
# Reload transcript with embeddings if already generated
transcript_f = 'black_w2v.pkl'
if exists(transcript_f):
    with open(transcript_f, 'rb') as f:
        transcript_w2v = pickle.load(f)

Have a look at the resulting `transcript_w2v` table created by the cells above. Check that the structure makes sense and confirm the total number of words in the transcript.

In [ ]:
# Inspect the transcript:


### Contextual word embeddings
In the second case, we'll extract contextual word embeddings from an autoregressive (or "causal") large language model (LLM) called GPT-2 ([Radford et al., 2019](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)). GPT-2 relies on the Transformer architecture to sculpt the embedding of a given word based on the preceding context. The model is composed of a repeated circuit motif—called "self-attention heads"—by which the model can "attend" to representations of previous words in the context window when determining the meaning of the current word. This GPT-2 implementation is composed of 12 layers, each of which contains 12 attention heads that influence the embedding as it proceeds to the subsequent layer. The embeddings at each layer of the model comprise 768 features and the context window includes the preceding 1024 tokens. Note that certain words will be broken up into multiple tokens; we'll need to use GPT-2's "tokenizer" to convert words into the appropriate tokens. GPT-2 has been (pre)trained on large corpora of text according to a simple self-supervised objective function: predict the next word (token) based on the prior context.

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

# Initialize tokenizer and model
model_name = 'gpt2'
n_features = 768

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Print out model architecture
model.eval()

In the Markdown cell below, answer the following questions:
1. What does the `50257` correspond to?
2. What does the `1024` correspond to?
3. What does the `768` correspond to?
4. What does the `12` in `12 x GPT2Block` correspond to?

*Your response here!*

Running the model on our transcript will be faster if GPUs are available.

In [ ]:
# Get device for running model (e.g. MacOS 'mps')
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

# You may need to force CPU on Mac, but try with 'mps' first
# device = 'cpu'

print(f"Using {device} device")

Now, we reload our original transcript. We'll run the tokenizer to convert the words into subword tokens for input to GPT-2. We need to keep track of the word indices when we run the tokenizer; we'll get an embedding for each token, but we'll want to recombine (i.e. average) embeddings for words split into multiple tokens. These tokens are supplied to GPT-2 as integer IDs corresponding to words the GPT-2 vocabulary (which contains approximately 50,000 words). Make sure you know what each line of code is doing.

In [ ]:
# Reload in transcript CSV file
transcript_f = 'encling-data/black_transcript.csv'
transcript_gpt2 = pd.read_csv(transcript_f)

# Insert explicit index column for reference
transcript_gpt2.insert(0, 'word_index', transcript_gpt2.index.values)

# Tokenize words into lists of tokens
transcript_gpt2['token'] = transcript_gpt2.word.apply(tokenizer.tokenize)

# "Explode" lists of token subwords into long format
transcript_gpt2 = transcript_gpt2.explode('token', ignore_index=True)

# Convert tokens to token IDs for input to model
transcript_gpt2['token_id'] = transcript_gpt2.token.apply(tokenizer.convert_tokens_to_ids)

Since our transcript contains more tokens than can be contained in GPT-2's context window, we'll aggregate the tokens into multiple `samples` matching the width of the context window (1024 tokens) and spanning the full length of the transcript. On some systems, we may be able to use the `accelerate` package to speed up model inference.

In [ ]:
# Convert all token IDs into list
token_ids = transcript_gpt2.token_id.tolist()

# Extract context window width for model
max_len = tokenizer.model_max_length

# Compile into lists of tokens within each context window
samples = []
token_ids = torch.tensor(transcript_gpt2.token_id.tolist(), dtype=torch.long)
samples.append(token_ids[0:max_len])
for i in range(max_len+1, len(token_ids)+1):
    samples.append(token_ids[i-max_len:i])

In [ ]:
from accelerate import Accelerator

# Initialize accelerator and free memory
accelerator = Accelerator()
accelerator.free_memory()

# Send model to device
model = model.to(device)

Finally, we'll use a PyTorch `DataLoader` to supply token IDs to the model in batches and extract the embeddings. In addition to the embeddings, we'll also extract several other features of potential interest from the model. As GPT-2 proceeds through the text, it generates a probability distribution (the `logits` extracted below) across all words in the vocabulary with the goal of correctly predicting the next word. We can use this probability distribution to derive other features of the model's internal computations. We'll extract the following features from GPT-2:
 
* `embeddings`: the 768-dimensional contextual embedding capturing the meaning of the current word
* `top_guesses`: the highest probability word GPT-2 predicts for the current word
* `ranks`: the rank of the correct word given probabilities across the vocabulary
* `true_probs`: the probability at which GPT-2 predicted the current word
* `entropies`: how the uncertain GPT-2 was about the current word
  * low entropy indicates that the probability distribution was "focused" on certain words
  * high entropy indicates the  probability distribution was more uniform/dispersed across words

*Note:* This upcoming cell may take a few minutes to run on a GPU (and longer on a CPU).

In [ ]:
# Set a batch size for the data loader
batch_size = 4

# Extract a late-intermediate layer from GPT-2
layer = 8

# Extract embeddings and other model features
embeddings = []
top_guesses = []
ranks = []
true_probs = []
entropies = []
with torch.no_grad():
    data_loader = torch.utils.data.DataLoader(samples, batch_size=batch_size,
                                              shuffle=False)

    # Loop through samples and extract embeddings
    for i, batch in enumerate(tqdm(data_loader)):
        output = model(batch.to(device), output_hidden_states=True)
        logits = output.logits  # torch.Size([2, 1024, 50257])
        states = output.hidden_states[layer]

        # Extract all embeddings/features for first context window
        if i == 0:
            true_ids = batch[0, :]
            brange = list(range(len(true_ids)-1))
            logits_order = logits[0].argsort(descending=True, dim=-1)
            batch_top_guesses = logits_order[:-1, 0]
            batch_ranks = torch.eq(logits_order[:-1],
                                   true_ids.reshape(-1,1)[1:].to(device)).nonzero()[:, 1]
            batch_probs = logits[0, :-1].softmax(-1)
            batch_true_probs = batch_probs[brange, true_ids[1:]]
            batch_entropy = torch.distributions.Categorical(probs=batch_probs).entropy()
            batch_embeddings = states[0]

            top_guesses.append(batch_top_guesses.numpy(force=True))
            ranks.append(batch_ranks.numpy(force=True))
            true_probs.append(batch_true_probs.numpy(force=True))
            entropies.append(batch_entropy.numpy(force=True))
            embeddings.append(batch_embeddings.numpy(force=True))
            
            # Reset if there are samples remaining in this batch
            if batch.size(0) == 1:
                continue
            logits = logits[1:]
            states = states[1:]
            batch = batch[1:]

        # Extract embeddings/features for last word in subsequent windows
        true_ids = batch[:, -1]
        brange = list(range(len(true_ids)))
        logits_order = logits[:, -2, :].argsort(descending=True)  # batch x vocab_size
        batch_top_guesses = logits_order[:, 0]
        batch_ranks = torch.eq(logits_order, true_ids.reshape(-1,1).to(device)).nonzero()[:, 1]
        batch_probs = torch.softmax(logits[:, -2, :], dim=-1)
        batch_true_probs = batch_probs[brange, true_ids]
        batch_entropy = torch.distributions.Categorical(probs=batch_probs).entropy()
        batch_embeddings = states[:, -1, :]

        top_guesses.append(batch_top_guesses.numpy(force=True))
        ranks.append(batch_ranks.numpy(force=True))
        true_probs.append(batch_true_probs.numpy(force=True))
        entropies.append(batch_entropy.numpy(force=True))
        embeddings.append(batch_embeddings.numpy(force=True))

We'll recompile the features we extracted from GPT-2 into our transcript and save it for easier loading later on. We'll also summarize how accurately GPT-2 was able to predict upcoming words.

In [ ]:
# Compile outputs into transcript (logit derivatives must be shifted by 1)
transcript_gpt2.loc[1:, 'rank'] = np.concatenate(ranks)
transcript_gpt2.loc[1:, 'true_prob'] = np.concatenate(true_probs)
transcript_gpt2.loc[1:, 'top_pred'] = np.concatenate(top_guesses)
transcript_gpt2.loc[0, 'top_pred'] = tokenizer.bos_token_id
transcript_gpt2.loc[1:, 'entropy'] = np.concatenate(entropies)
transcript_gpt2['embedding'] = [e for e in np.vstack(embeddings)]

# Reduce size of transcript
transcript_gpt2 = transcript_gpt2.astype({'word_index': 'int32', 'onset': 'float32',
                                      'offset': 'float32', 'token_id': 'int32',
                                      'rank': 'float32', 'true_prob': 'float32',
                                      'top_pred': 'int32', 'entropy': 'float32'}, copy=False)

# Convert model's top predictions from token IDs to tokens
transcript_gpt2['top_pred'] = transcript_gpt2.top_pred.apply(tokenizer.convert_ids_to_tokens)

# Print out top-1 and top-10 word prediction accuracy
print(f"Top-1 accuracy: {(transcript_gpt2['rank'] == 0).mean():.3f}")
print(f"Top-10 accuracy: {(transcript_gpt2['rank'] < 10).mean():.3f}")

# Save transcript with embeddings using pickle
with open('black_gpt2.pkl', 'wb') as f:
    pickle.dump(transcript_gpt2, f)

Take a moment to think about and understand what the "top-1 accuracy" and "top-10 accuracy" numbers mean here.

In [ ]:
# Reload transcript with embeddings if already generated
transcript_f = 'black_gpt2.pkl'
if exists(transcript_f):
    with open(transcript_f, 'rb') as f:
        transcript_gpt2 = pickle.load(f)

Have a look at the resulting transcript with embeddings and make sure it matches your intuitions. Specifically, check the first 10 tokens using `transcript_gpt2.iloc[0:10]`. What word had the lowest entropy? (Put differently, what word was the model the least uncertain about before it occurred?) What did the model most strongly expect for this token and why?

In [ ]:
# Inspect the transcript:


*Your response here!*

## Estimating and evaluating encoding models
In the following section, we'll use the embeddings extracted above to construct our encoding model. The goal of the encoding model is to evaluate how well we can predict brain activity from our representation of the stimulus in a given feature space ([Naselaris et al., 2011](https://doi.org/10.1016/j.neuroimage.2010.07.073)); in this case, our word embeddings encode linguistic content of the spoken story. We'll use linear regression to estimate a mapping from the word embeddings to the fMRI time series at each cortical parcel. Our model is very wide—containing hundreds or thousands of features or predictors (i.e. columns)—relative to the number of samples (i.e. TRs), so we are prone to overfitting. We'll rely on two fundamental principles of machine learning to mitigate the risks of overfitting. First, we'll introduce a *regularization* penalty (i.e. a particular kind of bias) into our regression equation to constrain how well the model (over)fits to the training data; ridge regression effectively "squeezes" the regression weights to reduce overfitting. Second, we will use *out-of-sample prediction* to evaluate the performance of our model; that is, we'll use cross-validation to train the model on subsets of the data and test the model on non-overlapping subsets of data.

### Construct predictor matrix
First, we need to resample our embeddings to ensure we have only vector per TR; the number of samples in our targets `Y` (parcelwise BOLD time series) and our predictors `X` must match. In this study, the fMRI data were acquired with 1.5-second TRs. For TRs with multiple embeddings (e.g. multiple words or tokens), we'll average the embeddings; for TRs with no embeddings (e.g. moments of silence), we'll insert zero vectors matching the length of the embeddings.

In [ ]:
# Function to average embeddings per TR
def construct_predictors(transcript_df, n_features, stim_dur, tr=1.5):

    # Find total number of TRs
    stim_trs = np.ceil(stim_dur / tr)

    # Add column to transcript with TR indices
    transcript_df['TR'] = transcript_df.onset.divide(tr).apply(np.floor).apply(int)
    
    # Compile the words within each TR
    words_per_tr = transcript_df.groupby('TR')['word'].apply(list)
    
    # Average the embeddings within each TR
    embeddings_per_tr = transcript_df.groupby('TR')['embedding'].mean()
    
    # Loop through TRs
    words_trs = []
    embeddings_trs = []
    for t in np.arange(stim_trs):
        if t in words_per_tr:
            words_trs.append(words_per_tr[t])
    
            # Fill in empty TRs with zero vectors
            if embeddings_per_tr[t] is not np.nan:
                embeddings_trs.append(embeddings_per_tr[t])
            else:
                embeddings_trs.append(np.zeros(n_features))
        else:
            words_trs.append([])
            embeddings_trs.append(np.zeros(n_features))
    
    embeddings = np.vstack(embeddings_trs)
    return embeddings

# word2vec embeddings are 300-dimensional
X_w2v = construct_predictors(transcript_w2v, 300, 800, tr=1.5)

# GPT-2 embeddings are 768-dimensional
X_gpt2 = construct_predictors(transcript_gpt2, 768, 800, tr=1.5)

Visualize both matrices of model features using `matshow`. Make sure you understand what the rows and columns correspond to (i.e., label your axes properly). You may want to use `zscore` from `scipy.stats` to mean-center and rescale the variance of each feature (column) prior to plotting the matrices (`vmin=-3` and `vmax=3` should work well with z-scored features).

In [ ]:
# Visualize word embedding predictor matrices:


The fMRI data were acquired the 8 TRs of fixation both before and after the story stimulus. We'll trim these TRs off to ensure the fMRI data match the predictors.

In [ ]:
# Trim fMRI data to match embeddings
start_trs = 8
end_trs = 8

assert start_trs + X_w2v.shape[0] + end_trs == func_parcels.shape[0]

Y_parcels = func_parcels[start_trs:-end_trs]

Check the shape of both sets of model features (`X_w2v` and `X_gpt2`) and the brain data (`Y_parcels`). The number of samples must match between the model features and our target variables for regression.

In [ ]:
# Check the shape of X and Y matrices:


### Ridge regression
Next, we'll use ridge regression to predict the activity at each parcel from the word embeddings. For this section, we'll consider each feature space (i.e. word2vec, GPT-2) in isolation. We'll use a split-half outer cross-validation scheme where we train the model on half of the story and test the model on the other half. To search for the best-performing regularization parameter $\lambda$ (`alpha` in scikit-learn convention), we'll perform 5-fold inner cross-validation within each training set using `KernelRidgeCV`; this will select the best parameter setting from the inner cross-validation fold within the training half to predict the test half. Higher `alpha` values increase regularization and reduce overfitting. Within each cross-validation fold, we'll apply two transforms: `StandardScaler` will be used to mean-center or z-score each column of the predictor matrix within the test set, then apply that transformation to the training set; `Delayer` will horiztonally stack lagged versions our predictor matrix to account for the hemodynamic lag. We use Himalaya's `KernelRidgeCV` (rather than scikit-learn's `RidgeCV`) because the multi-delayed version of the predictor matrix will be considerably wider than the number of fMRI samples. We'll combine these transforms and the estimator into a scikit-learn `Pipeline` that will run the whole analysis. This analysis qualitatively reproduces one of the core results from Huth and colleagues ([2016](https://doi.org/10.1038/nature17637)). Make sure you know what each line of code is doing!

In [ ]:
from sklearn.preprocessing import StandardScaler
from voxelwise_tutorials.delayer import Delayer
from sklearn.model_selection import KFold
from himalaya.kernel_ridge import KernelRidgeCV
from sklearn.pipeline import make_pipeline

# Split-half outer and inner cross-validation
outer_cv = KFold(n_splits=2)
inner_cv = KFold(n_splits=5)

# Mean-center each feature (columns of predictor matrix)
scaler = StandardScaler(with_mean=True, with_std=True)

# Create delays at 3, 4.5, 6, 7.5 seconds (1.5 s TR)
delayer = Delayer(delays=[2, 3, 4, 5])

# Ridge regression with alpha grid and nested CV
alphas = np.logspace(1, 10, 10)
ridge = KernelRidgeCV(alphas=alphas, cv=inner_cv)

# Chain transfroms and estimator into pipeline
pipeline = make_pipeline(scaler, delayer, ridge)

We'll run ridge regression separately for each feature space separately. Run the encoding analysis on the `word2vec` embeddings first and have a look at the results, then re-run the whole pipeline with the `GPT-2` embeddings. In the following cell, we loop through the outer cross-validation folds, fit the pipeline within each fold, then generate predictions. We use the model weights estimated from the training data to predict the brain activity from word embeddings for the test data. The `himalaya` implementation makes this surprisingly fast!

In [ ]:
# Select embeddings from one of the models
X = X_gpt2

# Loop through outer folds and estimate model
Y_predicted = []
for train, test in outer_cv.split(Y_parcels):
    
    # Fit pipeline with transforms and ridge estimator
    pipeline.fit(X[train],
                 Y_parcels[train])
    
    # Compute predicted response
    predicted = pipeline.predict(X[test])
    Y_predicted.append(predicted)
    
# Restack first and second half predictions
Y_predicted = np.vstack(Y_predicted)

Check the shape of the model-based predictions and make sure you understand the dimensions.

In [ ]:
# Check the model-based predictions:


To evaluate the predictions of our model, we quantify the similarity between the predicted brain activity and the actual brain activity for the test data. Keeping with conventions in the literature, we use Pearson correlation to assess the match between predicted and actual brain activity. Use `correlation_score` from `himalaya.scoring` to compute the Pearson correlation between the model-predicted (`Y_predicted`) and actual test (`Y_parcels`) time series for each parcel. Note that in this simplified approach we stack the model-predicted test sets (both of which are generated based on a separate training set) and run one correlation across both the stacked test sets; alternatively, we could compute the correlation for each test set in the cross-validation loop above.

In [ ]:
# Evaluate predictions using correlation between predicted and actual time series:


Print out the mean and maximum encoding performance values (out-of-sample correlations) across all parcels. Using `histplot` from seaborn (`sns`), create a simple histogram to visualize the correlation scores across all parcels.

In [ ]:
# Print mean and maximum encoding performance across parcels:

# Plot a histogram of prediction performance values:


We can also introspect certain attributes of our fitted encoding model. For example, we may want to examine, the best alpha values selected from hyperparameter search, or we may want to recover the weight vectors estimated for a given training set.

In [ ]:
# Introspect fitted pipeline model for alphas and weights
ridge_fitted = pipeline['kernelridgecv']
best_alphas = ridge_fitted.best_alphas_
weights = ridge_fitted.get_primal_coef(ridge_fitted.X_fit_)

For illustrative purposes, let's plot the actual and predicted time series for an example parcel. Use the same code you used to `plot` the time series from example parcel 195 above to plot the actual time series; then, plot the model-predicted time series for this parcel on top of the actual time series in a different color. You may want to z-score the actual and model-predicted time series for visualization (as the correlation we use to quantify model performance intrinsically z-scores the inputs). Can you visually see a match between the time series?

In [ ]:
# Plot predicted and actual response for example parcel:


Finally, let's look at a map of the results! Use the `masker`'s `.inverse_transform()` method to convert the parcel-level correlation scores back to into a brain image for visualization. Use `plot_stat_map` to visualize the brain map. I recommend using `vmax=.4`, `threshold=.1`, and consider visualizing both `cut_coords=(-55, -24, 9)` and `cut_coords=(5, -60, 33)` (I also used the `RdYlBu_r` color map).

In [ ]:
# Invert masker transform to project onto brain:

# Plot encoding performance correlations on brain:

# Plot correlations to visualize posterior medial cortex:


***Run it back!*** Before you proceed to the next section, go back to the cell where we picked one particular model—e.g., `X = X_w2v`. Switch to the other model (`X = X_gpt2`) and re-run the same set of cells up to this point. Take a moment to appreciate the difference between how each model performs separately. Note, however, that running to models separately and comparing their peformance is not a very sophisticated version of model comparison, because the two models will likely be correlated with each other (i.e., the features will be collinear across models). In the following section, we introduce a more advanced model comparison framework.

### Banded ridge regression
In the previous section, we fit separate models for the word2vec and GPT-2 embeddings. However, these embeddings likely encode a lot of similar information, making it difficult to compare the feature spaces. One way to compare feature spaces is to fit both feature spaces jointly, allowing them both to vie for variance in the brain activity—then evaluate their predictions separately. Our first thought might be to horizontally stack the word2vec and GPT-2 features, then refit the model in the same as the previous section. The problem with this approach is that our encoding model will find a single hyperparameter for both feature spaces. If the feature spaces are qualitatively different from each other, the hyperparameter will likely be unfairly biased toward one feature space. For example, if one feature space is very high-dimensional (i.e. wide) and the other is not, the fitting procedure will likely find a strong penalty term to regularize the wide predictors—but this hyperparameter may overly "squeeze" the lower-dimensional predictors. To solve this problem, we'll use Himalaya's implementation of *banded ridge regression* to estimate a separate regularization parameter for each feature space (i.e. band) in the joint predictor matrix ([Nunez-Elizalde et al., 2019](https://doi.org/10.1016/j.neuroimage.2019.04.012); [Dupré la Tour et al., 2022](https://doi.org/10.1016/j.neuroimage.2022.119728)). The first step is to horizontally-stack our predictors for both feature space; we'll use `slice`s to keep track of which columns belong to which feature space.

In [ ]:
# Horizontal-stack both embeddings to create joint model
X_joint = np.hstack([X_w2v, X_gpt2])

# Get the widths of each feature band
width_w2v = X_w2v.shape[1]
width_gpt2 = X_gpt2.shape[1]

# Set up slices for each feature band
slice_w2v = slice(0, width_w2v)
slice_gpt2 = slice(width_w2v, width_w2v + width_gpt2)

Print the shape of the joint predictor matrix, as well as both of the slicers, and make sure the numbers make sense to you.

In [ ]:
# Check the shape of the joint matrix and slices:


We'll set up the usual cross-validation scheme here. We need to take special care to apply the usual transforms—especially the kernelization step—to each set of predictors separately. We'll build a pipeline containing the necessary transforms, then use `himalaya`'s `ColumnKernelizer` apply this pipeline separately to both sets of predictors according to the `slice`s we defined.

In [ ]:
from himalaya.kernel_ridge import Kernelizer, ColumnKernelizer

# Split-half outer and inner cross-validation
outer_cv = KFold(n_splits=2)
inner_cv = KFold(n_splits=5)

# Make pipeline with kernelizer for each feature space
column_pipeline = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    Delayer(delays=[2, 3, 4, 5]),
    Kernelizer(kernel="linear"),
)

# Compile joint column kernelizer
column_kernelizer = ColumnKernelizer(
    [('word2vec', column_pipeline, slice_w2v),
     ('gpt2', column_pipeline, slice_gpt2)])

We'll use `MultipleKernelRidgeCV` to implement banded ridge regression with hyperparameter search. Since banded ridge regression applies separate penalty terms to each feature space, we need to search over combinations of hyperparameters. This can become very computationally expensive, so we'll use random search to find a good combination of regularization parameters. We initialize the model and link it up to our `ColumnKernelizer` pipeline.

In [ ]:
from himalaya.kernel_ridge import MultipleKernelRidgeCV

# Ridge regression with alpha grid and nested CV
solver = 'random_search'
n_iter = 20
alphas = np.logspace(1, 10, 10)
solver_params = dict(n_iter=n_iter, alphas=alphas)

# Banded ridge regression with column kernelizer
banded_ridge = MultipleKernelRidgeCV(kernels="precomputed", solver=solver,
                                     solver_params=solver_params, cv=inner_cv)

# Chain transfroms and estimator into pipeline
pipeline = make_pipeline(column_kernelizer, banded_ridge)

First, similarly to the previous exercise, we'll fit the joint model, generate a single set of predictions based on the combined feature spaces, and evaluate the quality of these predictions by computing the correlation between actual and predicted time series. This gives us an overall idea of how well the joint model predicts brain activity. Based on the code you used in previous section, inside the outer cross-validation loop, use the `pipeline`'s `.fit()` method to train the model (on the training samples from the model and target) and the `.predict()` method to generate predictions for the test set.

In [ ]:
# Loop through outer folds and estimate model
Y_predicted = []
for train, test in outer_cv.split(Y_parcels):
    
    # Fit the pipeline on X and Y training samples:
    # YOUR CODE HERE
    
    # Compute predicted response for X test samples and append:
    # YOUR CODE HERE
    
# Restack first and second half predictions:
# YOUR CODE HERE

Use `correlation_score` to compute model performance values per parcel like you did in the previous section. Print out the mean and maximum correlation scores along with a histogram across parcels.

In [ ]:
# Evaluate predictions using correlation between predicted and actual time series:

# Print mean and maximum encoding performance across parcels:

# Plot a histogram of prediction performance values:


Based on your code from the previous section, using the masker's `inverse_transform()` method to convert the parcel-level correlation scores into a brain image and visualize the model performance map using `plot_stat_map` (using the same arguments suggested above). This is not the end of the story, though! Next, we'll look at the relative contributions of each feature space...

In [ ]:
# Invert masker transform to project onto brain:

# Plot encoding performance correlations on brain:

# Plot correlations to visualize posterior medial cortex:


### Comparing feature spaces
Previously we estimated the joint model containing both word2vec and GPT-2 embeddings and evaluated the performance of the combined feature spaces. However, the more important question is typically: How do these feature spaces compare to one another? Here, we fit the joint model in the same way as the previous section—but critically we generate predictions separately for each feature space (using `split=True` in the `.predict()` method of the pipeline culminating in `MultipleKernelRidgeCV`). Remember that we generate model-based predictions by multiplying the weight vector estimated from the training set with the predictors from the test set. In this approach, we effectively zero-out weights for the feature-space(s)-of-no-interest to quantify how much one feature space contributes to performance.

In [ ]:
# Loop through outer folds and estimate model
Y_predicted = []
for train, test in outer_cv.split(Y_parcels):
    
    # Fit pipeline with transforms and ridge estimator
    pipeline.fit(X_joint[train],
                 Y_parcels[train])
    
    # Compute predicted response
    predicted = pipeline.predict(X_joint[test], split=True)
    Y_predicted.append(predicted)
    
# Restack first and second half predictions
Y_predicted = np.concatenate(Y_predicted, axis=1)

Use `correlation_score_split` from `himalaya.scoring` to easily compute correlation scores for the predictions based on each feature space. This will give you separate scores for word2vec and GPT-2. 

In [ ]:
# Compute correlation between predicted and actual for split predictions:


Let's look at the results! First, print out the mean and maximum correlation scores for both word2vec and GPT-2.

In [ ]:
# Print mean and maximum correlation scores for word2vec and GPT-2:


Finally, use the masker's `inverse_transform()` method to convert both sets of results back into two separate model performance maps. Plot these model performance maps using the procedure above (`plot_stat_map`) separately for the word2vec feature band and the GPT-2 feature band. How do the models compare?

In [ ]:
# Invert masker transform to project word2vec results onto brain:

# Plot word2vec encoding performance correlations on brain:

# Plot correlations to visualize posterior medial cortex:


In [ ]:
# Invert masker transform to project GPT-2 results onto brain:

# Plot GPT-2 encoding performance correlations on brain:

# Plot correlations to visualize posterior medial cortex:


Write a brief reflection (a couple sentences) about what you learned from these analyses. Consider the following points in your response:

* Consider which model performs best and in which brain areas—why do you think this model is superior?

* Can you think of any ways to even more stringently compare the two models?

* How do these encoding model performance maps compare to the contrast maps from Lab 1 and the ISC maps from Lab 2?

#### Reflection
*Your response here!*

### References
* Dupré la Tour, T., Eickenberg, M., Nunez-Elizalde, A. O., & Gallant, J. L. (2022). Feature-space selection with banded ridge regression. *NeuroImage*, *264*, 119728. https://doi.org/10.1016/j.neuroimage.2022.119728

* Dupré la Tour, T., Visconti di Oleggio Castello, M., & Gallant, J. L. (2025). The voxelwise encoding model framework: a tutorial introduction to fitting encoding models to fMRI data. *Imaging Neuroscience*, *3*, imag_a_00575. https://doi.org/10.1162/imag_a_00575

* Huth, A. G., De Heer, W. A., Griffiths, T. L., Theunissen, F. E., & Gallant, J. L. (2016). Natural speech reveals the semantic maps that tile human cerebral cortex. *Nature*, *532*(7600), 453–458. 

* Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. *arXiv*. https://doi.org/10.48550/arXiv.1301.3781

* Naselaris, T., Kay, K. N., Nishimoto, S., & Gallant, J. L. (2011). Encoding and decoding in fMRI. *NeuroImage*, *56*(2), 400-410. https://doi.org/10.1016/j.neuroimage.2010.07.073

* Nastase, S. A., Liu, Y.-F., Hillman, H., Zadbood, A., Hasenfratz, L., Keshavarzian, N., Chen, J., Honey, C. J., Yeshurun, Y., Regev, M., Nguyen, M., Chang, C. H. C., Baldassano, C., Lositsky, O., Simony, E., Chow, M. A., Leong, Y. C., Brooks, P. P., Micciche, E., Choe, G., Goldstein, A., Vanderwal, T., Halchenko, Y. O., Norman, K. A., & Hasson, U. (2021). The "Narratives" fMRI dataset for evaluating models of naturalistic language comprehension. *Scientific Data*, *8*, 250. https://doi.org/10.1038/s41597-021-01033-3

* Nunez-Elizalde, A. O., Huth, A. G., & Gallant, J. L. (2019). Voxelwise encoding models with non-spherical multivariate normal priors. *NeuroImage*, *197*, 482-492. https://doi.org/10.1016/j.neuroimage.2019.04.012

* Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language models are unsupervised multitask learners. *OpenAI Blog*. https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf

* Schaefer, A., Kong, R., Gordon, E. M., Laumann, T. O., Zuo, X. N., Holmes, A. J., Eickhoff, S. B., & Yeo, B. T. (2018). Local-global parcellation of the human cerebral cortex from intrinsic functional connectivity MRI. *Cerebral Cortex*, *28*(9), 3095–3114. https://doi.org/10.1093/cercor/bhx179

* Zada, Z., Goldstein, A. Y., Michelmann, S., Simony, E., Price, A., Hasenfratz, L., Barham, E., Zadbood, A., Doyle, W., Friedman, D., Dugan, P., Melloni, L., Devore, S., Flinker, A., Devinsky, O., Hasson, U.\*, & Nastase, S. A.\* (2024). A shared model-based linguistic space for transmitting our thoughts from brain to brain in natural conversations. *Neuron*, *112*(18), 3211–3222. https://doi.org/10.1016/j.neuron.2024.06.025

* Zada, Z., Nastase, S. A., Speer, S., Mwilambwe-Tshilobo, L., Tsoi, L., Burns, S., Falk, E., Hasson, U., & Tamir, D. (2025). Linguistic coupling between neural systems for speech production and comprehension during real-time dyadic conversations. *Neuron*, *114*(4), 774–787. https://doi.org/10.1016/j.neuron.2025.11.004